In [1]:
import pandas as pd 
import numpy as np 
import requests
import json
import dotenv
import os

import sys
sys.tracebacklimit = 0

In [2]:
dotenv.load_dotenv()
spotify_client_id = os.getenv('spotify_clientid')
spotify_client_secret = os.getenv('spotify_clientsecret')


url = 'https://accounts.spotify.com/api/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded',}
data = {
    'grant_type': 'client_credentials',
    'client_id': spotify_client_id,
    'client_secret': spotify_client_secret,
}

response = requests.post(url, headers=headers, data=data)

token_info = response.json()
access_token = token_info['access_token']

In [3]:
artist_id = "4YRxDV8wJFPHPTeXepOstw" # Arijit Singh
# artist_id = "2ae6PxICSOZHvjqiCcgon8" # SPB
# artist_id = "2oSONSC9zQ4UonDKnLqksx" # Atif Aslam
# artist_id = "1Xyo4u8uXC1ZmMpatF05PJ" # The Weeknd


options = "appears_on"
limit = "50"
url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}'
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers) 
total_album_no = json.loads(response.content)['total']

album_dict = {}
print(np.arange(0,total_album_no,50))
for i in np.arange(0,total_album_no,50):
    print(i)
    offset = i
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}&offset={offset}'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers) 

    df = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    album_dict[i] = df
album_df = pd.concat(album_dict.values())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
response

<Response [429]>

In [4]:
album_df['Album_ID'] = album_df['external_urls.spotify'].apply(lambda x: x.split("/")[-1])
album_df['Producer'] = album_df['artists'].apply(lambda x: x[0]['name'])
album_df['Producer_ID'] = album_df['artists'].apply(lambda x: x[0]['id'])
album_df['name'] = album_df['name'].apply(lambda x: x.replace("(Original Motion Picture Soundtrack)",""))
album_df.to_csv("albums.csv")
album_df


NameError: name 'album_df' is not defined

In [5]:
import time
all_albums = {}

for i,album_id in enumerate(album_df['Album_ID'].unique()):
    print(i,len(album_df))
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    try:
        response = requests.get(url, headers=headers) 
        df_t = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    except:
        
        print("Sleeping for 3 seconds")
        time.sleep(3)
        response = requests.get(url, headers=headers) 
        df_t = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)


    df_t['Album_ID'] = album_id



    df_t['Artist Song'] = df_t['artists'].apply(lambda x: artist_id in str(x))
    all_albums[album_id] = df_t

df_tracks = pd.concat(all_albums.values())
df_tracks.to_csv("tracks.csv")

NameError: name 'album_df' is not defined

In [6]:
json.loads(response.text)

{'href': 'https://api.spotify.com/v1/albums/6FWOmmRuTYal4b2UzIu2Ru/tracks?offset=0&limit=20',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kkQthS9OLpK4UqNWYqoVl'},
     'href': 'https://api.spotify.com/v1/artists/2kkQthS9OLpK4UqNWYqoVl',
     'id': '2kkQthS9OLpK4UqNWYqoVl',
     'name': 'Jeet Gannguli',
     'type': 'artist',
     'uri': 'spotify:artist:2kkQthS9OLpK4UqNWYqoVl'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/5IhbEmJKxDv7LBDBAsrOfE'},
     'href': 'https://api.spotify.com/v1/artists/5IhbEmJKxDv7LBDBAsrOfE',
     'id': '5IhbEmJKxDv7LBDBAsrOfE',
     'name': 'Subha Dutta',
     'type': 'artist',
     'uri': 'spotify:artist:5IhbEmJKxDv7LBDBAsrOfE'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/4kq8z3vydHjPDggxb16ErB'},
     'href': 'https://api.spotify.com/v1/artists/4kq8z3vydHjPDggxb16ErB',
     'id': '4kq8z3vydHjPDggxb16ErB',
     'name': 'Nakash Aziz',
     'type': 'artist',
     'uri':

In [ ]:
artist_songs = df_tracks[df_tracks['Artist Song']==True]
artist_songs = artist_songs[['artists','duration_ms','id','name','Album_ID']].rename(columns = {'name':"Track_Name",
                                                                                                'artists':"Track_Artists"})

album_details = album_df[['total_tracks','name','release_date','artists','Album_ID','Producer','Producer_ID']].rename(columns = {'name':'Album_Name',
                                                                                                                                'artists':"Album_Artists"})

final = artist_songs.merge(album_details, on = "Album_ID", how = "left", indicator=True)
print(final._merge.value_counts())

def get_track_pop(track_id):
    url = f'https://api.spotify.com/v1/tracks/{track_id}'
    try: return json.loads(requests.get(url, headers=headers).text)['popularity']
    except: return np.nan

final['Popularity'] = final['id'].apply(get_track_pop)
final.to_csv("All_Details_Full.csv", index = False)
final

_merge
both          2072
left_only        0
right_only       0
Name: count, dtype: int64


,Track_Artists,duration_ms,id,Track_Name,Album_ID,total_tracks,Album_Name,release_date,Album_Artists,Producer,Producer_ID,_merge
0,[{'external_urls': {'spotify': 'https://open.s...,261974,56zZ48jdyY2oDXHVnwg5Di,Tum Hi Ho,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
1,[{'external_urls': {'spotify': 'https://open.s...,304899,6UgcN95w7vQxkR8sEFmwHG,Chahun Main Ya Naa,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
2,[{'external_urls': {'spotify': 'https://open.s...,306511,55eUtE9v3uatR0y0NijM6N,Hum Mar Jayenge,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
3,[{'external_urls': {'spotify': 'https://open.s...,266069,3E9ryvrKP3hUWX79PAakH2,Meri Aashiqui,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
4,[{'external_urls': {'spotify': 'https://open.s...,214470,06zJUeKU5KFy50Cho6pDcr,Aasan Nahin Yahan,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
...,...,...,...,...,...,...,...,...,...,...,...,...
2067,[{'external_urls': {'spotify': 'https://open.s...,317413,0zciOy8kgToHnOc7m3RrAQ,Rishton Ke Saare Manzar,0vLCAjawZPbMtyhR6TcGED,10,Dastak - A Bouquet Of Ghazals,2015-01-01,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2068,[{'external_urls': {'spotify': 'https://open.s...,317348,21Nv3TlVXps943IGFWuJ8S,Rishton Ke Saare Manzar,35MkN8qsxx5KEELxxISsak,7,Celebrating World Music Day With Indipop Chart...,2015-06-18,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2069,[{'external_urls': {'spotify': 'https://open.s...,50749,3zk5QbltmuZWH8VapbdP4N,Phir Aur Kya Chahiye - Urban Drill Mix,2obuaVKDXzbuTNU9Tx0HfK,1,Phir Aur Kya Chahiye (Urban Drill Mix),2023-08-11,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2070,[{'external_urls': {'spotify': 'https://open.s...,200247,32t0LPcifv1tX4sGwrhBGE,"Hayi Hayi (From ""Dohchay"")",1GKO9IYiDGCCcwWNNV0JGV,13,Dance To The Tunes Turn This Party Up Vol-1,2020-09-23,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both


In [8]:
final_graph = final.copy()
final_graph = final_graph[final_graph['Producer']!="Various Artists"]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Best" not in x.title())]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Top" not in x.title())]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Hits" not in x.title())]

final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Remix" not in x.title())]
final_graph = final_graph[final_graph['Track_Name'].apply(lambda x: "Remix" not in x.title())]
final_graph = final_graph[final_graph['Track_Name'].apply(lambda x: "Mix" not in x.title())]
print(final_graph)
final_graph.to_csv("All_Details.csv", index = False)


                                          Track_Artists  duration_ms  \
0     [{'external_urls': {'spotify': 'https://open.s...       261974   
1     [{'external_urls': {'spotify': 'https://open.s...       304899   
2     [{'external_urls': {'spotify': 'https://open.s...       306511   
3     [{'external_urls': {'spotify': 'https://open.s...       266069   
4     [{'external_urls': {'spotify': 'https://open.s...       214470   
...                                                 ...          ...   
1947  [{'external_urls': {'spotify': 'https://open.s...       188813   
1971  [{'external_urls': {'spotify': 'https://open.s...       219193   
2000  [{'external_urls': {'spotify': 'https://open.s...       210619   
2003  [{'external_urls': {'spotify': 'https://open.s...       192400   
2064  [{'external_urls': {'spotify': 'https://open.s...       270624   

                          id                            Track_Name  \
0     56zZ48jdyY2oDXHVnwg5Di                             Tum Hi H

In [12]:
import lyricsgenius
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
import time

genius_access_token = os.getenv('geniusclientaccesstoken')
genius = lyricsgenius.Genius(genius_access_token)
final_graph = pd.read_csv("All_Details.csv")

In [19]:
first_n = 50
for i in range(final_graph.shape[0]):
    if i==first_n: break
    print(i)
    song_dets = final_graph.iloc[i,:].to_dict()
    # print(song_dets.keys())
    lyrics = ""
    query = song_dets['Track_Name'] + " "
    try:
        try:
            lyrics = genius.search_song(query + " Romanized").lyrics
        except:
            lyrics = genius.search_song(query).lyrics
            lyrics = transliterate(lyrics, sanscript.DEVANAGARI, sanscript.ITRANS)
    except: continue
        
    final_graph.loc[i,"Lyrics"] = lyrics.split("Lyrics")[-1].title()
    time.sleep(1)
final_graph = final_graph.iloc[:first_n, :]

0
Searching for "Tum Hi Ho  Romanized"...
Searching for "Tum Hi Ho "...
1
Searching for "Chahun Main Ya Naa  Romanized"...
Searching for "Chahun Main Ya Naa "...
2
Searching for "Hum Mar Jayenge  Romanized"...
Searching for "Hum Mar Jayenge "...
3
Searching for "Meri Aashiqui  Romanized"...
Searching for "Meri Aashiqui "...
4
Searching for "Aasan Nahin Yahan  Romanized"...
Searching for "Aasan Nahin Yahan "...
5
Searching for "Milne Hai Mujhse Aayi  Romanized"...
Searching for "Milne Hai Mujhse Aayi "...
6
Searching for "Ilahi  Romanized"...
Searching for "Ilahi "...
7
Searching for "Dilliwaali Girlfriend  Romanized"...
Searching for "Dilliwaali Girlfriend "...
8
Searching for "Kabira (Encore)  Romanized"...
Searching for "Kabira (Encore) "...
9
Searching for "Satranga  Romanized"...
Searching for "Satranga "...
10
Searching for "Soulmate  Romanized"...
Searching for "Soulmate "...
11
Searching for "Hamari Adhuri Kahani (Title Track)  Romanized"...
Searching for "Hamari Adhuri Kahani (

In [16]:
final_graph.to_csv("All_Details_with_Lyrics.csv", index = False)

In [16]:
### TOP TRACKS OF ARTIST
artist_id = "4YRxDV8wJFPHPTeXepOstw" # Arijit Singh
# artist_id = "2ae6PxICSOZHvjqiCcgon8" # SPB
# artist_id = "2oSONSC9zQ4UonDKnLqksx" # Atif Aslam
# artist_id = "1Xyo4u8uXC1ZmMpatF05PJ" # The Weeknd

location = "IN"
url = f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks?market={location}'
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers) 
top_tracks = json.loads(response.content)

# album_dict = {}
# print(np.arange(0,total_album_no,50))
# for i in np.arange(0,total_album_no,50):
#     print(i)
#     offset = i
#     url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}&offset={offset}'
#     headers = {'Authorization': f'Bearer {access_token}'}
#     response = requests.get(url, headers=headers) 

#     df = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
#     album_dict[i] = df
# album_df = pd.concat(album_dict.values())

In [21]:
top_tracks['tracks']

[{'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3PpjqSKBer3obW6lDflOBx'},
     'href': 'https://api.spotify.com/v1/artists/3PpjqSKBer3obW6lDflOBx',
     'id': '3PpjqSKBer3obW6lDflOBx',
     'name': 'Ram Sampath',
     'type': 'artist',
     'uri': 'spotify:artist:3PpjqSKBer3obW6lDflOBx'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/4YRxDV8wJFPHPTeXepOstw'},
     'href': 'https://api.spotify.com/v1/artists/4YRxDV8wJFPHPTeXepOstw',
     'id': '4YRxDV8wJFPHPTeXepOstw',
     'name': 'Arijit Singh',
     'type': 'artist',
     'uri': 'spotify:artist:4YRxDV8wJFPHPTeXepOstw'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/7vCsTTACb2iT4fvBMnwp5w'},
     'href': 'https://api.spotify.com/v1/artists/7vCsTTACb2iT4fvBMnwp5w',
     'id': '7vCsTTACb2iT4fvBMnwp5w',
     'name': 'Prashant Pandey',
     'type': 'artist',
     'uri': 'spotify:artist:7vCsTTACb2iT4fvBMnwp5w'}],
   'external_urls': 